In [1]:
# 필요한 라이브러리 임포트
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(OPENAI_API_KEY[:2])


c:\Users\winds\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


up


In [4]:
# 0단계: 문서 로드
loader = PyPDFLoader('../../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()

print(f"문서 개수: {len(documents)}")
print(f"첫 번째 문서 길이: {len(documents[0].page_content)}")
print(f"첫 번째 문서 미리보기:\n{documents[0].page_content[:500]}...")


문서 개수: 109
첫 번째 문서 길이: 0
첫 번째 문서 미리보기:
...


In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        # 법률 사례는 1200-1800자 권장
    chunk_overlap=300,      # 사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     # 법률 문서 섹션 구분자
        "\n【쟁점사항】",     # 쟁점 부분 구분
        "\n【처리경위】",     # 처리 과정 구분
        "\n【처리결과】",     # 결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
texts = text_splitter.split_documents(documents)

print(f"분할된 청크 개수: {len(texts)}")
print(f"첫 번째 청크 길이: {len(texts[0].page_content)}")
print(f"첫 번째 청크 미리보기:\n{texts[0].page_content[:300]}...")

분할된 청크 개수: 104
첫 번째 청크 길이: 32
첫 번째 청크 미리보기:
콘텐츠분쟁조정 법리 연구 2부
- 타 분쟁조정사례 조사 -...


In [6]:
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

In [8]:
vectorstore = FAISS.from_documents(texts, embeddings)

# 검색기 설정
retriever = vectorstore.as_retriever(
    search_type="similarity",        
    search_kwargs={"k": 5}          
)



In [10]:
llm = ChatUpstage(
    model="solar-pro",
    base_url="https://api.upstage.ai/v1",
    temperature=0.2
)



In [ ]:
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               # 실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  # 한계 인정

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)


In [12]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                        
    chain_type="stuff",             
    retriever=retriever,            
    chain_type_kwargs={"prompt": prompt}, 
    return_source_documents=True    
)

In [15]:

def test_qa_system(question):
    print(f"\n{'='*80}")
    print(f"질문: {question}")
    print(f"{'='*80}")
    
    try:
        result = qa_chain({"query": question})
        print(f"\n답변:\n{result['result']}")
        
        print(f"\n참조 문서 개수: {len(result['source_documents'])}")
        print("\n참조 문서 미리보기:")
        for i, doc in enumerate(result['source_documents'], 1): 
            print(f"\n[참조 문서 {i}]")
            print(f"내용: {doc.page_content[:200]}...")
            
    except Exception as e:
        print(f"오류 발생: {e}")




In [20]:
pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]


In [21]:
for q in pythontest_questions:
    test_qa_system(q)
    print("=*"*200)


질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

답변:
### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 사안  

#### 1. **사례 기반 분석**  
제시된 사례집에 따르면, 시스템 오류로 인한 아이템 복구 문제는 다음과 같은 요소에 따라 처리 결과가 달라집니다:  
- **계정 소유권 확인**: 피신청인(게임사)이 계정 명의자와 실소유자를 엄격히 구분하는 경우, 명의자가 아닌 이용자의 복구 요청은 거부될 수 있습니다(2006년 사례).  
- **오류 입증 가능성**: 시스템 오류가 객관적으로 확인되지 않거나, 다른 이용자에게 동일한 문제가 발생하지 않은 경우 복구 요구가 거절될 수 있습니다(2009년 사례).  
- **게임사의 복구 정책**: 프로그램 오류가 인정되면 복구가 이루어지기도 하나(2006년 프로그램 오류 사례), 데이터 확인 지연 등으로 신속한 처리가 어려울 수 있습니다.  

#### 2. **법적 근거**  
- **민법 제250조(도품·유실물 특례)**:  
  - 아이템이 해킹 또는 시스템 오류로 소멸된 경우, 게임사는 "유실물 반환의무"를 질 수 있으나, **금전적 가치가 없는 디지털 아이템**은 해당 조항 적용이 제한적일 수 있습니다.  
  - 단, 게임머니 등 현금성 아이템은 예외적 적용 가능성이 있습니다.  
- **전자상거래법 및 약관 규제법**:  
  - 게임사가 약관에서 "시스템 오류로 인한 책임은 지지 않는다"는 조항을 명시했더라도, **불공정약관**에 해당할 경우 무효될 수 있습니다(약관 규제법 제6조).  
  - 예: "모든 책임은 이용자에게 있다"는 조항은 소비자 보호 측면에서 문제될 수 있음.  

#### 3. **실무적 해결 방안**  
**(1) 1단계: 게임사에 공식 문의 및 증거 수집**  
- **계정 소유권 확인**: 명의자와 실소유자가 다를 경우, 명의자 변경을 먼저 요청하세요.  
-

In [23]:
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

for q in pythontest_questions:
    print(classify_dispute_type(q))

게임
이러닝
이러닝
게임
기타
